In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import gymnasium as gym
import pygame
import random
from collections import deque

# See environment
Just sample a random game and view the results

In [384]:
# Create the environment
env = gym.make('CartPole-v1', render_mode="human")

# Reset the environment to get the initial state
state = env.reset()

# Run a random policy for a few episodes
for episode in range(5):
    state, info = env.reset()
    done = False
    total_reward = 0

    while not done:
        env.render()  # Render the environment

        # Take a random action
        action = env.action_space.sample()

        # Step the environment with the chosen action
        state, reward, done, truncated, info = env.step(action)
        total_reward += reward

        if done or truncated:
            break

    print(f"Episode {episode + 1}: Total Reward: {total_reward}")

env.close()

Episode 1: Total Reward: 22.0
Episode 2: Total Reward: 11.0
Episode 3: Total Reward: 17.0
Episode 4: Total Reward: 24.0
Episode 5: Total Reward: 16.0


# Create a replay buffer
The replay buffer stores previous experiences like, states, actions, rewards, next states and if the simulation ended (dones)

In [3]:
class ReplayBuffer:
    def __init__(self, buffer_size, batch_size, seed=42, device='cpu'):
        self.memory = deque(maxlen=buffer_size)
        self.batch_size = batch_size
        self.seed = random.seed(seed)
        self.device = device

    def add(self, experience):
        self.memory.append(experience)

    def sample(self):
        experiences = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states, dones = zip(*experiences)
        return (
            torch.tensor(states, dtype=torch.float32, device=self.device),
            torch.tensor(actions, dtype=torch.int64, device=self.device),
            torch.tensor(rewards, dtype=torch.float32, device=self.device),
            torch.tensor(next_states, dtype=torch.float32, device=self.device),
            torch.tensor(dones, dtype=torch.uint8, device=self.device)
        )

    def __len__(self):
        return len(self.memory)


# Create Q-Network
Lets start by adding a simple Q-network

In [4]:
class QNetwork(nn.Module):
    def __init__(self, state_size, action_size, hidden_size=42, dropout=0.2):
        super().__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, action_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# Create an agent
Lets create an agent that can be trained.


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [402]:
class DQNAgent:
    def __init__(self, state_size, action_size, seed):
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)

        # Q-Network
        self.qnetwork_local = QNetwork(state_size, action_size, hidden_size=512, dropout=0.0).to(device)
        self.qnetwork_target = QNetwork(state_size, action_size, hidden_size=512, dropout=0.0).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=1e-3)

        # Replay memory
        self.memory = ReplayBuffer(buffer_size=int(1e5), batch_size=64, seed=seed, device=device)
        self.t_step = 0

    def step(self, state, action, reward, next_state, done):
        self.memory.add((state, action, reward, next_state, done))

        self.t_step = (self.t_step + 1) % 4
        if self.t_step == 0:
            if len(self.memory) > self.memory.batch_size:
                experiences = self.memory.sample()
                self.learn(experiences, gamma=0.99)

    def act(self, state, eps=0.):
        state = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()

        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))

    def learn(self, experiences, gamma):
        states, actions, rewards, next_states, dones = experiences

        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        Q_targets = rewards.unsqueeze(1) + (gamma * Q_targets_next * (1 - dones).unsqueeze(1))

        Q_expected = self.qnetwork_local(states).gather(1, actions.unsqueeze(1))

        loss = F.mse_loss(Q_expected, Q_targets)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        self.soft_update(self.qnetwork_local, self.qnetwork_target, tau=1e-3)

    def soft_update(self, local_model, target_model, tau):
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)


In [403]:
#env_name = 'CartPole-v0' #Max episode length is 200
env_name = 'CartPole-v1' #Max episode length is 500


In [404]:
env = gym.make(env_name)
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size, seed=42)

n_episodes = 10000
max_t = 1500
eps_start = 1.0
eps_end = 0.05
eps_decay = 0.995

scores = []
scores_window = deque(maxlen=100)
eps = eps_start
for i_episode in range(1, n_episodes + 1):
    state, info = env.reset()
    score = 0
    for t in range(max_t):
        action = agent.act(state, eps)
        next_state, reward, done, truncated, info = env.step(action)
        agent.step(state, action, reward, next_state, done)
        state = next_state
        score += reward
        if done:
            break
    scores_window.append(score)
    scores.append(score)
    eps = max(eps_end, eps_decay * eps)

    print(f"Episode {i_episode}\tAverage Score: {np.mean(scores_window):.2f}")

    if np.mean(scores_window) >= 500*0.9:
        print(f'\nEnvironment solved in {i_episode-100:d} episodes!\tAverage Score: {np.mean(scores_window):.2f}')
        torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
        break

env.close()

Episode 1	Average Score: 11.00
Episode 2	Average Score: 15.50
Episode 3	Average Score: 20.33
Episode 4	Average Score: 24.75
Episode 5	Average Score: 28.20
Episode 6	Average Score: 31.17
Episode 7	Average Score: 28.86
Episode 8	Average Score: 26.88
Episode 9	Average Score: 25.56
Episode 10	Average Score: 23.90
Episode 11	Average Score: 23.36
Episode 12	Average Score: 23.08
Episode 13	Average Score: 23.69
Episode 14	Average Score: 23.07
Episode 15	Average Score: 22.93
Episode 16	Average Score: 23.69
Episode 17	Average Score: 23.06
Episode 18	Average Score: 22.50
Episode 19	Average Score: 22.21
Episode 20	Average Score: 21.60
Episode 21	Average Score: 21.52
Episode 22	Average Score: 23.00
Episode 23	Average Score: 22.43
Episode 24	Average Score: 22.29
Episode 25	Average Score: 22.16
Episode 26	Average Score: 22.23
Episode 27	Average Score: 21.93
Episode 28	Average Score: 22.64
Episode 29	Average Score: 22.24
Episode 30	Average Score: 22.33
Episode 31	Average Score: 22.13
Episode 32	Averag

In [395]:
def visualize_agent(agent, env, n_episodes=5):
    for i_episode in range(1, n_episodes + 1):
        state, info = env.reset()
        done = False
        score = 0
        while not done:
            env.render()
            action = agent.act(state)
            next_state, reward, done, truncated, info = env.step(action)
            state = next_state
            score += reward
            if done:
                break
        print(f"Episode {i_episode}\tScore: {score}")
    env.close()

In [400]:
# Visualize the agent
env = gym.make('CartPole-v1', render_mode="human")
visualize_agent(agent, env)

Episode 1	Score: 277.0
Episode 2	Score: 260.0
Episode 3	Score: 255.0
Episode 4	Score: 262.0
Episode 5	Score: 264.0


In [18]:
env = gym.make('CartPole-v1', render_mode="human")


In [382]:
state, info = env.reset()
state
env.render()